In [18]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,  StandardScaler, OneHotEncoder
import pickle
from tensorflow.keras.models import load_model  


In [19]:
#load the models 

#Loding trained model
model = load_model('model.h5')

#loading OHE model
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

#loading LabelEncoded model
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender  = pickle.load(file)

#loading scaler model
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [20]:
input_data = {
    "CreditScore": 650,
    "Geography": "Germany",
    "Gender" : "Male",
    "Age": 35,
    "Tenure": 5,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 0,
    "EstimatedSalary": 75000
}


In [24]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,Germany,Male,35,5,60000,2,1,0,75000


In [21]:
#OHE 
onehot_geo = onehot_encoder_geo.transform([[input_data["Geography"]]]).toarray()
onehot_geo_df = pd.DataFrame(onehot_geo, columns = onehot_encoder_geo.get_feature_names_out(["Geography"]))
onehot_geo_df 

c:\Users\lenovo\Desktop\GenAI\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [25]:
#combine OHE with input data
input_df = pd.concat([input_df.drop("Geography",axis = 1), onehot_geo_df] ,axis = 1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,Male,35,5,60000,2,1,0,75000,0.0,1.0,0.0


In [26]:
#Label
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])


In [27]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,1,35,5,60000,2,1,0,75000,0.0,1.0,0.0


In [28]:
input_scaled = scaler.transform(input_df)
input_scaled


array([[-1.70986091e-02,  9.13247552e-01, -3.70568589e-01,
        -1.34471949e-03, -2.57811188e-01,  8.08436154e-01,
         6.49202671e-01, -1.02583358e+00, -4.42165451e-01,
        -9.98501123e-01,  1.72572313e+00, -5.76388018e-01]])

In [30]:
# Prediction
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step


array([[0.07777222]], dtype=float32)

In [32]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.077772215)

In [33]:
if prediction_proba > 0.5:
    print("Cstomer is likely to churn")
else:
    print("The customer is not likely to churn") 

The customer is not likely to churn
